In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd huggan/pytorch/cyclegan
!pip install wandb
!wandb login
!accelerate config

In [ ]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os



hyperparams_cyclegan = {
    'decay_epoch':[10] , #100
    'lr':[0.0002], 
    'n_residual_blocks':[9], 
    
    'lambda_cyc':[10.0], 
    'lambda_id':[5.0], 
    'beta1':[0.5], 
    'beta2':[0.999], 
    'batch_size':[8],
    
    'dataset_name':['Chris1/sim2real_gta5_to_cityscapes'],#['Chris1/NFT_cryptopunks_to_bored_apes'],  

    'channels':[3], 
    'checkpoint_interval':[10], 
    #'cpu':[False],     
    'epoch':[0], 
    #'fp16':[False], 
    'image_size':[64], 
    'mixed_precision':['no'], 
    'model_name':[None],
    'num_epochs':[20], 
    'num_workers':[8], 
    'organization_name':['huggan'], 
    'push_to_hub':[False], 
    'pytorch_dump_folder_path':[None], 
    'sample_interval':[100]
                     }



In [ ]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)



for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
        call(call_params)
        